In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy.interpolate import interp1d
import os

# Import required libraries
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import Adam

In [113]:
def create_sequences(X, y, time_steps=24):
    seq_indices = np.arange(0, X.shape[0] - time_steps + 1)
    xs = [X[i:i+time_steps] for i in seq_indices]
    ys = y[time_steps - 1:]
    return xs, ys

#this function won't work quite right--I think it checks the overall number of nas 
def filter_out_nas(list_of_dfs, max_consecutive_nas):
    index_fil = []
    for i, df in enumerate(list_of_dfs):
        observed_column = df['observed']
        # Check if there are any NAs at the beginning or end
        first_na = observed_column.isna().values[0]
        last_na = observed_column.isna().values[-1]
        # Check if the number of consecutive NAs is within the threshold
        num_consecutive_nas = np.sum(pd.isna(observed_column))
        # use run length  
        if num_consecutive_nas <= max_consecutive_nas and not first_na and not last_na:
            index_fil.append(i)
    return index_fil



In [114]:
# Prepare data
train_data_fp = "C:/Users/ttrefoni/Documents/tt_pm25_tuning/data/TrainData/TrainData"
td_files = os.listdir(train_data_fp)

x_list = []
y_list= []

#read in full df for scaling 
for file in td_files:
    i_df = pd.read_csv(os.path.join(train_data_fp, file))
    i_df['file_id'] = os.path.basename(file)
    x_list.append(i_df)  # Append each dataframe to x_list

# Concatenate all dataframes into one
full_df = pd.concat(x_list, ignore_index=True)
full_df 

,datetime,epa_pm25,date,temperature,humidity,pm25_cf_1,pm25_cf_1_b,pm25_cf_1_a,year,month,R2,calibrated,PearsonR,calv2,file_id
0,2021-04-15 22:00:00,6.0,2021-04-15 17:45:56.307692288,68.461538,26.307692,2.379615,2.279231,2.480000,2021,4,0.999831,4.811038,0.906382,4.811038,EPA1200~PA104240.csv
1,2021-04-15 23:00:00,6.0,2021-04-15 18:30:48.033333248,72.833333,25.000000,285.067167,267.172667,302.961667,2021,4,0.999831,121.371368,0.906382,121.371368,EPA1200~PA104240.csv
2,2021-04-16 00:00:00,6.0,2021-04-15 19:30:48.700000000,74.000000,25.666667,113.413167,107.516667,119.309667,2021,4,0.999831,63.850879,0.906382,63.850879,EPA1200~PA104240.csv
3,2021-04-16 01:00:00,4.0,2021-04-15 20:30:49.033333504,74.000000,26.000000,39.462500,37.122000,41.803000,2021,4,0.999831,26.366329,0.906382,26.366329,EPA1200~PA104240.csv
4,2021-04-16 02:00:00,5.0,2021-04-15 21:30:50.000000000,73.300000,25.800000,13.508167,12.777000,14.239333,2021,4,0.999831,11.425028,0.906382,11.425028,EPA1200~PA104240.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876836,2022-01-01 03:00:00,16.2,2021-12-31 22:29:52.900000000,57.566667,66.766667,16.765667,15.962667,17.568667,2022,1,0.996783,12.006813,0.850793,12.006813,EPA949~PA5276.csv
876837,2022-01-01 04:00:00,14.6,2021-12-31 23:29:53.900000000,57.833333,65.933333,15.928833,15.129667,16.728000,2022,1,0.996783,11.541068,0.850793,11.541068,EPA949~PA5276.csv
876838,2022-01-01 05:00:00,14.8,2022-01-01 00:29:54.866666752,58.666667,63.600000,14.932833,14.095667,15.770000,2022,1,0.996783,11.029596,0.850793,11.029596,EPA949~PA5276.csv
876839,2022-01-01 06:00:00,12.4,2022-01-01 01:29:55.800000000,59.000000,62.000000,19.009333,18.067333,19.951333,2022,1,0.996783,13.493770,0.850793,13.493770,EPA949~PA5276.csv


In [115]:
# Prepare data
train_data_fp = "C:/Users/ttrefoni/Documents/tt_pm25_tuning/data/TrainData/TrainData"
td_files = os.listdir(train_data_fp)

x_list = []
y_list= []
i_list=[]
#read in full df for scaling 
for file in td_files:
    i_df = pd.read_csv(os.path.join(train_data_fp, file))
    i_df['file_id'] = os.path.basename(file)
    i_list.append(i_df)  # Append each dataframe to x_list

# Concatenate all dataframes into one
full_df = pd.concat(i_list, ignore_index=True)
#create scaler 
scaler = StandardScaler()
scaler.fit(full_df[['pm25_cf_1', 'temperature', 'humidity']])

for file in td_files[0:1]:
    df_org = pd.read_csv(os.path.join(train_data_fp, file))
    df_org['file_id'] = os.path.basename(file)
    #drop extraneous columns
    df = df_org.drop(
        columns=['pm25_cf_1_b', 'date', 'pm25_cf_1_a', 'year', 'month', 'R2', 'calibrated', 'PearsonR', 'calv2'])
    # Convert the 'datetime_column' to datetime format
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S')
    
    #sort by date time 
    df = df.sort_values(by='datetime').reset_index(drop=True)
    df['observed'] = 1
    # Floor and ceiling the 'datetime' column
    earliest_date = df['datetime'].min()
    latest_date = df['datetime'].max()
    #add rows for hours not observed
    df = df.set_index('datetime').apply(lambda x: x.reindex(pd.date_range(min(x.index), max(x.index), freq='h')))
    #linearlly interpolate across gaps in observations 
    df[['epa_pm25', 'temperature', 'humidity', 'pm25_cf_1']] = df[
        ['epa_pm25', 'temperature', 'humidity', 'pm25_cf_1']].apply(lambda group: group.interpolate(method='linear'))
    #reset file_id column 
    df[['file_id']] = file
    
    #scale input based off range of original  
    x = df[['pm25_cf_1', 'temperature', 'humidity', 'observed']]
    #x.loc[:, ['pm25_cf_1', 'temperature', 'humidity']] = scaler.transform(x[['pm25_cf_1','temperature','humidity']])
    #seperate output 
    y = df[['epa_pm25']]
    time_steps = 24
    x_seq, y_seq = create_sequences(x, np.array(y), time_steps)
    #filter sequences to remove sequences with too many consecutive na values
    max_consec_nas = 3
    filtered_index = filter_out_nas(x_seq, max_consec_nas)
    x_fil = [x_seq[i] for i in filtered_index]
    y_fil = y_seq[filtered_index]
    print(file)
    
    #remove extra columns for X
    columns_to_remove = ['observed']
    #x_fil = [df.loc[:, df.columns != 'observed'] for df in x_fil]
    #x_fil= [df.reset_index(drop=True) for df in x_fil] 
    
    #append to overall list
    x_list.extend(x_fil)
    y_list.extend(y_fil)


AttributeError: 'numpy.float64' object has no attribute 'isna'

In [ ]:
np.random.seed(42)
samp = 0.7
index = np.random.choice(len(y_list), int(samp * len(y_list)), replace=False)

x_train = [x_list[i] for i in index]
y_train = [y_list[i][0] for i in index]
x_test = [x_list[i] for i in range(len(y_list)) if i not in index]
y_test =[y_list[i][0] for i in range(len(y_list)) if i not in index]

In [17]:
x_train_array = np.array(x_train)
y_train_array = np.array(y_train)
x_test_array = np.array(x_test)
y_test_array = np.array(y_test)

In [39]:
# Save to numpy arrays
np.save("C:/Users/ttrefoni/Documents/tt_pm25_tuning/data/updt_seq_npy_arrays_70_30_py/x_train.npy", x_train_array)
np.save("C:/Users/ttrefoni/Documents/tt_pm25_tuning/data/updt_seq_npy_arrays_70_30_py/y_train.npy", y_train_array)
np.save("C:/Users/ttrefoni/Documents/tt_pm25_tuning/data/updt_seq_npy_arrays_70_30_py/x_test.npy", x_test_array)
np.save("C:/Users/ttrefoni/Documents/tt_pm25_tuning/data/updt_seq_npy_arrays_70_30_py/y_test.npy", y_test_array)

In [108]:
x_list[224]

,pm25_cf_1,temperature,humidity,observed
2021-07-22 21:00:00,4.592667,116.933333,4.333333,1.0
2021-07-22 22:00:00,3.895833,118.633333,3.400000,1.0
2021-07-22 23:00:00,3.490333,117.833333,3.533333,1.0
2021-07-23 00:00:00,3.873500,114.066667,5.266667,1.0
2021-07-23 01:00:00,4.154000,109.566667,6.066667,1.0
2021-07-23 02:00:00,4.854000,105.533333,8.033333,1.0
2021-07-23 03:00:00,6.474500,101.433333,11.600000,1.0
2021-07-23 04:00:00,8.505000,96.500000,16.300000,1.0
2021-07-23 05:00:00,8.126833,93.200000,20.266667,1.0
2021-07-23 06:00:00,8.215500,89.533333,23.900000,1.0


In [ ]:
#drop extraneous columns
df = df_org.drop(columns=['pm25_cf_1_b', 'date', 'pm25_cf_1_a', 'year', 'month', 'R2', 'calibrated', 'PearsonR', 'calv2'])
# Convert the 'datetime_column' to datetime format
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S')

df = df.sort_values(by='datetime').reset_index(drop=True)
df['observed'] = 1

In [ ]:
# Step 1: Floor and ceiling the 'datetime' column
earliest_date = df['datetime'].min()
latest_date = df['datetime'].max()

df = df.set_index('datetime').apply(lambda x: x.reindex(pd.date_range(min(x.index), max(x.index), freq='h')))

In [ ]:
#linearlly interpolate
df[['epa_pm25', 'temperature', 'humidity', 'pm25_cf_1']] = df[['epa_pm25', 'temperature', 'humidity', 'pm25_cf_1']].apply(lambda group: group.interpolate(method='linear'))
#reset file_id column 
df[['file_id']]='placeholder'
#df[['file_id']]=file

In [ ]:
x = df[['pm25_cf_1', 'temperature', 'humidity','observed']]
scaler = StandardScaler()
x[['pm25_cf_1','temperature','humidity']] = scaler.fit_transform(x[['pm25_cf_1','temperature','humidity']])

y = df[['epa_pm25']]

In [ ]:
time_steps = 24
x_seq, y_seq = create_sequences(x, np.array(y), time_steps)

In [ ]:
column_values = [m['observed'] for m in x_seq]
column_values

In [ ]:
def filter_out_nas(list_of_dfs, max_consecutive_nas):
    index_fil = []
    for i, df in enumerate(list_of_dfs):
        observed_column = df['observed']
        # Check if there are any NAs at the beginning or end
        first_na = observed_column.isna().values[0]
        last_na = observed_column.isna().values[-1]
        # Check if the number of consecutive NAs is within the threshold
        num_consecutive_nas = np.sum(pd.isna(observed_column))
        if num_consecutive_nas <= max_consecutive_nas and not first_na and not last_na:
            index_fil.append(i)
    return index_fil

In [ ]:
max_consec_nas = 3
filtered_index = filter_out_nas(x_seq, max_consec_nas)
filtered_index[1:10]

In [ ]:
x_fil = [x_seq[i] for i in filtered_index]
y_fil = y_seq[filtered_index]
x_fil[230]

In [ ]:
np.random.seed(42)
samp = 0.7
index = np.random.choice(len(y_fil), int(samp * len(y_fil)), replace=False)

x_train = [x_fil[i] for i in index]
y_train = y_fil[index]
x_test = [x_fil[i] for i in range(len(y_fil)) if i not in index]
y_test = y_fil[[i for i in range(len(y_fil)) if i not in index]]
